# JOB OFFER'S DATA (part 3: merging and cleaning)

In [47]:
# Load nessesary libraries.
import pandas as pd
import re
from random import randint
from datetime import datetime, timedelta
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [31]:
# Define global variables.
label_value = ['registered_nurse', 'electrician', 'data_analyst']
csv_file_name = ['data_jobads_rn.csv', 'data_jobads_e.csv', 'data_jobads_da.csv']
data_frame = ['df1', 'df2', 'df3']
to_remove = ['salary', 'schedule', 'benefits', 'location', 'job type', 'office']
date_of_download = "January 10, 2024"
before_30_days = 'before December 11, 2023'
directory = r'C:\Users\temulenbd\OneDrive\Desktop\learn\github_repo\cct\capstone_project'

In [32]:
# Define a custom function to import a CSV file, add a label column, and return the DataFrame.
def import_and_label(csv, data_f, value):
    
    # Read the CSV file into a DataFrame.
    data_f = pd.read_csv(csv, index_col=None)
    
    # Add a new column 'label' with the specified value.
    data_f['label'] = value
    
    # eturn the modified DataFrame
    return data_f

In [33]:
# Call the import_and_label function for the each CSV file.
df1 = import_and_label(csv_file_name[0], data_frame[0], label_value[0])
df2 = import_and_label(csv_file_name[1], data_frame[1], label_value[1])
df3 = import_and_label(csv_file_name[2], data_frame[2], label_value[2])

# Concatenate the three DataFrames along the rows (axis=0) and reset the index.
df = pd.concat([df1, df2, df3], ignore_index=True)

# Check the DataFrame.
print(df.shape)
df.head(2)

(1034, 6)


,title,id,link,date,job_description,label
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Just posted,Silver Stream Healthcare Group offer great emp...,registered_nurse
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Hiring ongoing,Create A Better Future For Yourself !!\nRecrui...,registered_nurse


**'date' column**

In [34]:
# Define a custom function to replace 'Just posted' or 'Today' with 'January 10, 2024'.
def replace_just_posted_today(date):
    global date_of_download
    return date_of_download if 'Just posted' in str(date) or 'Today' in str(date) else date

# Define a custom function extract and replace with relevant information.
def remove_elements_with_colon(date):
    date = replace_just_posted_today(date)
    
    if date and ('day' in str(date) or 'days' in str(date)):
        days_ago = int(re.search(r'(\d+) (?:day|days) ago', str(date)).group(1))
        new_date = datetime.strptime('January 10, 2024', '%B %d, %Y') - timedelta(days=days_ago)
        return new_date.strftime('%B %d, %Y')
    else:
        return date

In [35]:
# Apply changes.
df['date'] = df['date'].apply(replace_just_posted_today)
df['date'] = df['date'].replace(['not available', 'Hiring ongoing', None], 'unknown')
df['date'] = df['date'].replace(['Posted 30+ days ago'], before_30_days)
df['date'] = df['date'].apply(remove_elements_with_colon)

# Assuming df is your DataFrame and 'date' is the column
date_counts = df['date'].value_counts()

# Print the counts for each unique value in the 'date' column
print(date_counts)

before December 11, 2023    511
unknown                      84
January 08, 2024             71
December 20, 2023            46
January 05, 2024             45
December 22, 2023            37
January 10, 2024             30
January 03, 2024             30
January 09, 2024             28
January 04, 2024             25
January 06, 2024             18
December 13, 2023            14
December 19, 2023            12
December 14, 2023            11
December 12, 2023            10
December 21, 2023            10
December 23, 2023             8
December 16, 2023             7
December 30, 2023             6
January 02, 2024              6
December 15, 2023             5
January 07, 2024              5
December 29, 2023             3
December 18, 2023             2
January 01, 2024              2
December 28, 2023             2
December 26, 2023             2
December 31, 2023             1
December 11, 2023             1
December 24, 2023             1
December 27, 2023             1
Name: da

**'job_description' column**

In [36]:
# Define the costum function to find and remove unnecessary or private information.
def remove_elements_with_colon(column_value, remove_value):
        
    # Convert the value to lowercase.
    column_value = column_value.lower()
    
    # Split each value of the column into a list using '\n' as a separator.
    elements = column_value.split('\n')
    
    # Find the index of the element containing given value.
    index_of_element = next((i for i, elements in enumerate(elements) if remove_value in elements and ':' in elements), None)
    
    # Check if the first conditions are present in the text.
    if index_of_element is not None:
        next_colon_index = next((j for j in range(index_of_element + 1, len(elements)) if ':' in elements[j]), None)
        
        # Add an extra condition to check the second conditions are present in the text.
        if next_colon_index is not None:
            del elements[index_of_element:next_colon_index]
            return '\n'.join(elements)
        else:
            return column_value
        
    else:
        return column_value

In [37]:
# Apply the function to the column.
for val_rem in to_remove:
    df['job_description'] = df['job_description'].apply(lambda x: remove_elements_with_colon(x, val_rem))

# Print the modified DataFrame.
print(df.iat[randint(0, 1034), 4])
print('---------')
print(df.iat[randint(0, 1034), 4])

operating for over 45 years, king & moffatt building services provide complete mechanical and electrical services across all sectors of the construction industry including industrial, logistics, food, pharmaceutical, data centre, mining, healthcare, commercial, hotel & leisure, high-end residential, public, and energy.

from design stage to the installation and the ongoing maintenance of their facilities, our valued and highly skilled workforce provide our clients with an efficient, reliable and best in class service.

a deloitte best managed company award winner for eight years running, we pride ourselves on being a full building services solutions provider and have successfully delivered multiple large-scale projects in ireland, the uk & mainland europe.

what you would do:

test electrical wiring, equipment, appliances, apparatus, and fixtures, using appropriate tools and record and report on all works. sign electrical installation certs.
diagnose malfunctioning systems, apparatus, 

**summarization of 'job_description' column long values**

In [38]:
# Define custom function to count words in a cell.
def count_words(cell):
    words = cell.split()
    return len(words)

In [39]:
# Apply the function to the job_description column.
df['word_count'] = df['job_description'].apply(lambda x: count_words(x))

# Check the changes.
df.head(2)

,title,id,link,date,job_description,label,word_count
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"January 10, 2024",silver stream healthcare group offer great emp...,registered_nurse,556
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,unknown,create a better future for yourself !!\nrecrui...,registered_nurse,250


In [40]:
# Load public-free fine tuning dataset from Datasets library.
billsum = load_dataset('billsum', split='ca_test')

# Split the dataset to test and train set.
billsum = billsum.train_test_split(test_size=0.2, seed=1)

# Check the train test values.
billsum['train'][0]
billsum['test'][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 13651 of the Business and Professions Code is amended to read:\n13651.\n(a) (1) Every service station in this state shall provide during operating hours, and make available at no cost to customers who purchase motor vehicle fuel, water, compressed air, and a gauge for measuring air pressure to the public for use in servicing any passenger vehicle, as defined in Section 465 of the Vehicle Code, or any commercial vehicle, as defined in Section 260 of the Vehicle Code, with an unladen weight of 6,000 pounds or less.\n(2) Every service station in this state shall display, at a conspicuous place on, at, or near the dispensing apparatus at least one clearly visible sign that shall read as follows: “CALIFORNIA LAW REQUIRES THIS STATION TO PROVIDE FREE AIR AND WATER FOR AUTOMOTIVE PURPOSES TO ITS CUSTOMERS WHO PURCHASE MOTOR VEHICLE FUEL. IF YOU HAVE A COMPLAINT NOTIFY THE STATION ATTENDANT AND/OR CAL

In [41]:
# Load t5 cokenizer to process text and summary.
checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [42]:
# Create a DataCollatorForSeq2Seq instance for sequence-to-sequence tasks.
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [43]:
# Load the ROUGE evaluation metric using the `evaluate` module.
rouge = evaluate.load("rouge")

# Define a custom function that passes predictions and labels to compute to calculate the ROUGE metric.
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [52]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="./Seq2SeqTraining",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/248 [00:00<?, ?it/s]

c:\Users\temulenbd\anaconda3\Lib\site-packages\transformers\generation\utils.py:1126: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 2.829505205154419, 'eval_rouge1': 0.1232, 'eval_rouge2': 0.034, 'eval_rougeL': 0.1022, 'eval_rougeLsum': 0.1023, 'eval_gen_len': 19.0, 'eval_runtime': 357.1417, 'eval_samples_per_second': 0.694, 'eval_steps_per_second': 0.045, 'epoch': 1.0}


  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 2.5532679557800293, 'eval_rouge1': 0.1344, 'eval_rouge2': 0.0464, 'eval_rougeL': 0.1111, 'eval_rougeLsum': 0.1111, 'eval_gen_len': 19.0, 'eval_runtime': 358.1858, 'eval_samples_per_second': 0.692, 'eval_steps_per_second': 0.045, 'epoch': 3.0}


  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 2.5373153686523438, 'eval_rouge1': 0.1372, 'eval_rouge2': 0.048, 'eval_rougeL': 0.1134, 'eval_rougeLsum': 0.1133, 'eval_gen_len': 19.0, 'eval_runtime': 353.4876, 'eval_samples_per_second': 0.702, 'eval_steps_per_second': 0.045, 'epoch': 4.0}
{'train_runtime': 9038.7018, 'train_samples_per_second': 0.438, 'train_steps_per_second': 0.027, 'train_loss': 3.0181601739698842, 'epoch': 4.0}


TrainOutput(global_step=248, training_loss=3.0181601739698842, metrics={'train_runtime': 9038.7018, 'train_samples_per_second': 0.438, 'train_steps_per_second': 0.027, 'train_loss': 3.0181601739698842, 'epoch': 4.0})

In [53]:
trainer.save_model('fine_tuned_t5_model_by_temuulen')

In [54]:
results = trainer.evaluate()

c:\Users\temulenbd\anaconda3\Lib\site-packages\transformers\generation\utils.py:1126: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?it/s]

In [56]:
metrics = results
print("Metrics:", metrics)

Metrics: {'eval_loss': 2.5373153686523438, 'eval_rouge1': 0.1372, 'eval_rouge2': 0.048, 'eval_rougeL': 0.1134, 'eval_rougeLsum': 0.1133, 'eval_gen_len': 19.0, 'eval_runtime': 359.2004, 'eval_samples_per_second': 0.69, 'eval_steps_per_second': 0.045, 'epoch': 4.0}


In [72]:
text = df.iat[532, 4]

In [73]:
text


"we’re looking for a registered nurse for a nursing home to care for our patients with compassion and one who can facilitate their speedy recovery. the candidate will be responsible for educating them and their families on prevention methods and healthy habits. ideally, the candidate should be a responsible, gentle and well-trained professional who can provide exceptional nursing care with minimal supervision whilst following health and safety guidelines faithfully and consistently. the goal is to promote the patient's well-being by providing high quality nursing care.\nmonitor patient’s condition and assess their needs to provide the best possible care.\nobserve and interpret patient’s symptoms and communicate them to physicians.\ncollaborate with physicians and nurses to devise individualized care plans for patients.\nperform routine procedures (blood pressure measurements, administering injections etc.) and fill in patient’s charts.\nadjust and administer patient’s medication(s) and

In [66]:
from transformers import pipeline

summarizer = pipeline("summarization", model="fine_tuned_t5_model_by_temuulen")

summary = summarizer(text, max_length=500, min_length=400, length_penalty=2.0, num_beams=4)[0]['summary_text']

print(summary)

Your max_length is set to 500, but your input_length is only 320. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=160)


ttm healthcare are currently recruiting a clinical nurse manager 2 theatre on behalf of an east dublin based hospital . the role is responsible for the provision of a quality service in line with standards of theatre nursing practice . as a key member of the senior management team, the position requires a strategic approach to the development of services and structures, embracing continuous quality improvement and the management of changes necessary to achieve organisational objectives. be registered in the general division of the register of nurses maintained by nmbi have at least five years nursing experience in the cnm2 in theatre, a full time permanent role working 37.5 hours a week . this is a . a health care . staff and staff in an east . dublin's a staff member, the role will demonstrate managerial and leadership skills and facilitate effective communication with colleagues in the hospital, the . ‘the . it requires . be a. the position . to be registered . in a ‘continued .. the

In [67]:
summarizer = pipeline("summarization", model="t5-small")

summary = summarizer(text, max_length=500, min_length=400, length_penalty=2.0, num_beams=4)[0]['summary_text']
print(summary)

Your max_length is set to 500, but your input_length is only 320. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=160)


ttm healthcare are currently recruiting a clinical nurse manager 2 theatre . role is responsible for the provision of a quality service in line with standards of theatre nursing practice . position requires a strategic approach to the development of services and structures . be registered in the general division of the register of nurses maintained by nmbi have at least five years relevant post-registration nursing experience (full-time or equivalent hours part-time) in an acute hospital setting . minimum of three years nursing experience in peri-operative theatre nursing.com . tp a nurse manager for a . the cnm 2 theatre working 37.5 hours a week and a key member of the senior management and the role will demonstrate managerial and leadership skills and the management of the . and the 'the management of nm2 in theatre (nmbi . of the main . nursing . in a post . or in n - post a recognised post- ni . post nbc . experience in an ad . have a nursing a minimum of 'full i . on a regular ba

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Example tokenized texts as strings
text1 = "ttm healthcare are currently recruiting a clinical nurse manager 2 theatre on behalf of an east dublin based hospital.\nthis is a full time permanent role working 37.5 hours per week.\n\nthe role:\nthe role is responsible for the provision of a quality service in line with standards of theatre nursing practice. as a key member of the senior management team, the cnm2 in theatre (general) will demonstrate managerial and leadership skills and facilitate effective communication with colleagues in the hospital. the position requires a strategic approach to the development of services and structures, embracing continuous quality improvement and the management of changes necessary to achieve organisational objectives.\n\nessential criteria:\nbe registered in the general division of the register of nurses maintained by nmbi\nhave at least five years recent relevant post-registration nursing experience (full-time or equivalent hours part-time) in an acute hospital setting and a minimum of three years nursing experience in peri-operative theatre nursing\nhave a recognised post-registration nursing course - higher diploma or post grad course, in peri-operative theatre nursing or equivalent or be in pursuit of same\n\nbenefits:\ncompetitive salary\nup to 5% pension contribution\nsick pay\nmaternity benefit\nfree parking\nan education support programme\ndevelopment opportunities\nopportunities for career progression\nsubsidised restaurant\nemployee assistance programme\nlife assurance\nfree parking\n\na full job description is available upon request.\nfor more information call louise on 015136740 or click apply with your most recent cv today\n\nttm healthcare solutions is an equal opportunities employer"
text2 = "person specification: qualifications:\nactive nmbi pin\npostgraduate qualification in cardiology/cardiovascular nursing preferred.\nadvanced health assessment desirable.\n\nrequired experience & skills:\nadvanced cardiovascular life support certification.\n3-5 years of cardiology nursing in the acute setting within the past 5-7 years.\nleadership experience required.\nproficient in speaking, reading, and writing grammatically correct english.\neffective time management skills.\n\nbenefits of working with three q:\nassistance with improving your cv.\ninterview preparation support.\na €250 referral campaign for recommending qualified candidates in the healthcare industry.\na €250 training bursary to support your professional development.\nafter-placement support ensures your continued success in the healthcare field."
text3 = "we’re looking for a registered nurse for a nursing home to care for our patients with compassion and one who can facilitate their speedy recovery. the candidate will be responsible for educating them and their families on prevention methods and healthy habits. ideally, the candidate should be a responsible, gentle and well-trained professional who can provide exceptional nursing care with minimal supervision whilst following health and safety guidelines faithfully and consistently. the goal is to promote the patient's well-being by providing high quality nursing care.\nmonitor patient’s condition and assess their needs to provide the best possible care.\nobserve and interpret patient’s symptoms and communicate them to physicians.\ncollaborate with physicians and nurses to devise individualized care plans for patients.\nperform routine procedures (blood pressure measurements, administering injections etc.) and fill in patient’s charts.\nadjust and administer patient’s medication(s) and provide treatments according to the physician’s orders.\ninspect the facilities and act to maintain excellent hygiene and safety (decontaminating equipment, sanitizing surfaces, preparing beds etc.).\nprovide instant medical care in emergencies.\nassist surgeons during operations.\nsupervise and train lpns and nursing assistants.\nfoster a supportive and compassionate environment to care for patients and their families.\nexpand their knowledge and capabilities by attending educational workshops, conferences etc.\nrequirements\nrequirements:\nregistered with the nmbi (nursing and midwifery board of ireland)\nat least 6 months-1 year post-registration experience in elderly care.\nexcellent communication and interpersonal skills.\nstamp 4 or valid irish work permit (eligible to work in ireland)\nproven experience as a registered nurse.\nexcellent knowledge of nursing care methods and procedures.\nexcellent knowledge of emergency care.\nin-depth knowledge of health and safety guidelines and procedures (sanitation, decontamination etc.) and willingness to follow them at all times.\na team player with excellent communication and interpersonal skills.\nresponsible and compassionate.\noutstanding organizational and multi-tasking skills.\npatient with excellent problem-solving skills.\nexperience:\nhome health: 1 year (required)\neducation:\nbachelor's (required)\nlicence:\nnursing and midwifery board of ireland registration (required)"

# Create CountVectorizer and fit-transform the texts
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([text1, text2, text3])

# Compute cosine similarity between the vectors
cosine_sim = cosine_similarity(X)

# Normalize the vectors before computing cosine similarity
normalized_X = normalize(X, norm='l2', axis=1)
cosine_sim_normalized = cosine_similarity(normalized_X)

print("Cosine Similarity Matrix:")
print(cosine_sim)
print("\nCosine Similarity Matrix (Normalized):")
print(cosine_sim_normalized)

Cosine Similarity Matrix:
[[1.         0.49612862 0.4358911 ]
 [0.49612862 1.         0.29630706]
 [0.4358911  0.29630706 1.        ]]

Cosine Similarity Matrix (Normalized):
[[1.         0.49612862 0.4358911 ]
 [0.49612862 1.         0.29630706]
 [0.4358911  0.29630706 1.        ]]


In [79]:
!pip install azure-ai-textanalytics==5.3.0

     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     -------------- ------------------------- 30.7/82.8 kB 1.3 MB/s eta 0:00:01
     ---------------------------- --------- 61.4/82.8 kB 656.4 kB/s eta 0:00:01
     -------------------------------------- 82.8/82.8 kB 771.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
     ---------------------------------------- 41.7/41.7 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/298.6 kB ? eta -:--:--
   --------- ------------------------------ 71.7/298.6 kB 1.9 MB/s eta 0:00:01
   -------------------- ------------------- 153.6/298.6 kB 1.8 MB/s eta 0:00:01
   ------------------------------------- -- 276.5/298.6 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 298.6/298.6 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/192.9 kB ? eta -:--:--
   ----------------------- ---------------- 112.6/192.9 kB 3.3 MB/s eta 0:0

In [81]:
key = '26b9cb9a3b884f789b1a98d4f0d79b60'
endpoint = 'https://temulenbd.cognitiveservices.azure.com/'

In [82]:
set LANGUAGE_KEY '26b9cb9a3b884f789b1a98d4f0d79b60'

SyntaxError: invalid syntax (2573901743.py, line 1)

In [11]:
import torch                
from transformers import AutoTokenizer, AutoModelWithLMHead

In [13]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')                        
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)   

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\temulenbd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\temulenbd\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

c:\Users\temulenbd\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
c:\Users\temulenbd\anaconda3\Lib\site-packages\transformers\models\auto\modeling_auto.py:1573: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for c

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
inputs = tokenizer.encode("summarize: " + text,
                          return_tensors='pt',
                          truncation=True)


NameError: name 'tokenizer' is not defined

In [ ]:
summary_ids = model.generate(inputs, max_length=500, min_length=300, length_penalty=5., num_beams=2)        

In [15]:
'REVISITING FEW-SAMPLE BERT FINE-TUNING'.lower()

'revisiting few-sample bert fine-tuning'

In [45]:
file_path = os.path.join(directory, 'data_jobads_final.csv')
    
# Export the DataFrame to CSV file.
df.to_csv(file_path, index=False)
    
print(f"The raw data was transformed and exported successfully as {file_path}.")

The raw data was transformed and exported successfully as C:\Users\temulenbd\OneDrive\Desktop\learn\github_repo\cct\capstone_project\data_jobads_final.csv.
